# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [3]:
import logging
import os
import csv

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from azureml.data.dataset_factory import TabularDatasetFactory
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset

from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION) 

SDK version: 1.21.0


In [4]:
#Testing the authentication using the Workspace method "from_config"
Workspace.from_config()

Workspace.create(name='quick-starts-ws-137654', subscription_id='6971f5ac-8af1-446e-8034-05acea24681f', resource_group='aml-quickstarts-137654')

In [5]:
#create workspace
ws = Workspace.from_config()

In [6]:
 #!pip install --upgrade --upgrade-strategy eager azureml-sdk[automl,widgets,notebooks]==1.21.0

In [7]:
 print("SDK version:", azureml.core.VERSION)

SDK version: 1.21.0


In [8]:
# Choose a name for the run history container in the workspace.
# NOTE: update these to match your existing experiment name
experiment_name = 'automl-experiment-1'
project_folder = './pipeline-project'

experiment = Experiment(ws, experiment_name)
experiment

Name,Workspace,Report Page,Docs Page
automl-experiment-1,quick-starts-ws-137654,Link to Azure Machine Learning studio,Link to Documentation


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [9]:


dataset = TabularDatasetFactory.from_delimited_files('https://www.openml.org/data/get_csv/16826755/phpMYEkMl')


dataset = dataset.to_pandas_dataframe()
dataset.describe()

,pclass,survived,sibsp,parch,fare
count,1309.000000,1309.000000,1309.000000,1309.000000,1308.000000
mean,2.294882,0.381971,0.498854,0.385027,33.295479
std,0.837836,0.486055,1.041658,0.865560,51.758668
min,1.000000,0.000000,0.000000,0.000000,0.000000
25%,2.000000,0.000000,0.000000,0.000000,7.895800
50%,3.000000,0.000000,0.000000,0.000000,14.454200
75%,3.000000,1.000000,1.000000,0.000000,31.275000
max,3.000000,1.000000,8.000000,9.000000,512.329200


In [10]:
###Just in case:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "auto-ml"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_DS12_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           min_nodes=1,
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.

Running


### Clean Data:

In [11]:
from train import clean_data

In [12]:
x, y = clean_data(dataset)

In [13]:
from sklearn.preprocessing import StandardScaler

In [14]:
# create scaler
variables = x.columns.tolist()

scaler = StandardScaler()
scaler.fit(x[variables]) 

x = scaler.transform(x[variables])

In [15]:
x = pd.DataFrame(x,columns=variables)


In [16]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x,y,
                                                    test_size = 0.2,
                                                    random_state=0)

In [17]:
#bring them together them again
dataset = pd.concat([x_train,y_train],axis=1)

In [18]:
dataset.head()

,pclass,age,sibsp,parch,fare,age_NA,fare_NA,sex_male,cabin_Missing,cabin_Rare,embarked_Q,embarked_Rare,embarked_S,title_Mr,title_Mrs,title_Rare,survived
1118,0.841916,-0.349075,-0.479087,-0.445,-0.490240,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0
44,-1.546098,0.891204,-0.479087,-0.445,1.956990,-0.501432,-0.02765,-1.344995,-1.853992,2.347858,-0.322040,-0.039118,-1.521159,-1.172894,-0.42592,-0.274947,1
1072,0.841916,-0.116523,-0.479087,-0.445,-0.493947,1.994288,-0.02765,0.743497,0.539377,-0.425920,3.105202,-0.039118,-1.521159,0.852592,-0.42592,-0.274947,0
1130,0.841916,-0.891698,-0.479087,-0.445,-0.493141,-0.501432,-0.02765,-1.344995,0.539377,-0.425920,-0.322040,-0.039118,0.657394,-1.172894,-0.42592,-0.274947,0
574,-0.352091,-0.039005,0.481288,-0.445,-0.237445,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0


In [19]:
#To train the model we need a TabularDataset and not a dataframe, therefore the current df will be converterd 
#into a TabularDataset:

#Convert the dataframe into a csv
local_path = 'prepared.csv'

#Save it locally
dataset.to_csv(local_path,index=None)

#Generate the a datastore object which is the the default datastore
datastore = ws.get_default_datastore()

In [20]:
#Upload the dataframe which was previosly converted into a csv
datastore.upload(src_dir='.', target_path='data')

Uploading an estimated of 14 files
Target already exists. Skipping upload for data/.amlignore
Target already exists. Skipping upload for data/.amlignore.amltmp
Target already exists. Skipping upload for data/automl3.ipynb.amltmp
Target already exists. Skipping upload for data/automl4.ipynb
Target already exists. Skipping upload for data/automl4.ipynb.amltmp
Target already exists. Skipping upload for data/prepared.csv
Target already exists. Skipping upload for data/pre_preprocess.py
Target already exists. Skipping upload for data/train.py
Target already exists. Skipping upload for data/__pycache__/pre_preprocess.cpython-36.pyc
Target already exists. Skipping upload for data/__pycache__/train.cpython-36.pyc
Uploading ./.ipynb_aml_checkpoints/automl4-checkpoint2021-1-6-19-16-2.ipynb
Uploaded ./.ipynb_aml_checkpoints/automl4-checkpoint2021-1-6-19-16-2.ipynb, 1 files out of an estimated total of 6
Uploading ./.ipynb_aml_checkpoints/automl4-checkpoint2021-1-6-19-27-51.ipynb
Uploaded ./.ipynb

$AZUREML_DATAREFERENCE_7153bfb72a6a4424914d388474887e94

In [21]:
#For the sake of checking; check the path
datastore.path()

$AZUREML_DATAREFERENCE_workspaceblobstore

In [22]:
#Now the uploaded file will be transformed into a Tabular dataset and store in a varible named 'training_dataset'
training_dataset = Dataset.Tabular.from_delimited_files(path= [(datastore,('data/prepared.csv'))])

In [23]:
#let's visualize the data:
training_dataset.to_pandas_dataframe().head()

,pclass,age,sibsp,parch,fare,age_NA,fare_NA,sex_male,cabin_Missing,cabin_Rare,embarked_Q,embarked_Rare,embarked_S,title_Mr,title_Mrs,title_Rare,survived
0,0.841916,-0.349075,-0.479087,-0.445,-0.490240,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0
1,-1.546098,0.891204,-0.479087,-0.445,1.956990,-0.501432,-0.02765,-1.344995,-1.853992,2.347858,-0.322040,-0.039118,-1.521159,-1.172894,-0.42592,-0.274947,1
2,0.841916,-0.116523,-0.479087,-0.445,-0.493947,1.994288,-0.02765,0.743497,0.539377,-0.425920,3.105202,-0.039118,-1.521159,0.852592,-0.42592,-0.274947,0
3,0.841916,-0.891698,-0.479087,-0.445,-0.493141,-0.501432,-0.02765,-1.344995,0.539377,-0.425920,-0.322040,-0.039118,0.657394,-1.172894,-0.42592,-0.274947,0
4,-0.352091,-0.039005,0.481288,-0.445,-0.237445,-0.501432,-0.02765,0.743497,0.539377,-0.425920,-0.322040,-0.039118,0.657394,0.852592,-0.42592,-0.274947,0


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [24]:
#Create the automl settings which will be used as argurments in the automl config
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'accuracy'
}

#Create the automl_config
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=training_dataset,
                             label_column_name="survived",   
                             path = project_folder,
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings
                            )

In [25]:
#New one
#Original code
# TODO: Submit your experiment
#remote_run = experiment.submit(automl_config)
automl_run = experiment.submit(automl_config,show_output=True)

Running on remote.
No run_configuration provided, running on auto-ml with default configuration
Running on remote compute: auto-ml
Parent Run ID: AutoML_504d70f0-c693-4ec3-bcf5-3860b75a3534

Current status: FeaturesGeneration. Generating features for the dataset.
Current status: ModelSelection. Beginning model selection.

****************************************************************************************************
DATA GUARDRAILS: 

TYPE:         Cross validation
STATUS:       DONE
DESCRIPTION:  Each iteration of the trained model was validated through cross-validation.
              
DETAILS:      
+---------------------------------+
|Number of folds                  |
+=================================+
|3                                |
+---------------------------------+

****************************************************************************************************

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, a

In [26]:
#Additional Run Details
from azureml.widgets import RunDetails

RunDetails(automl_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

In [27]:
#Get generic outputs from the automl_run
automl_run.get_output()

(Run(Experiment: automl-experiment-1,
 Id: AutoML_504d70f0-c693-4ec3-bcf5-3860b75a3534_38,
 Type: azureml.scriptrun,
 Status: Completed),
 Pipeline(memory=None,
          steps=[('datatransformer',
                  DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                  feature_sweeping_config=None,
                                  feature_sweeping_timeout=None,
                                  featurization_config=None, force_text_dnn=None,
                                  is_cross_validation=None,
                                  is_onnx_compatible=None, logger=None,
                                  observer=None, task=None, working_dir=None)),
                 ('prefittedsoftvotingclassifier',...
                                                                                                 n_estimators=100,
                                                                                                 n_jobs=1,
                      

In [28]:
#Get the best model outputs
best_automl_run, best_model = automl_run.get_output()


In [29]:
#Get the best model id
print(best_automl_run.id)

AutoML_504d70f0-c693-4ec3-bcf5-3860b75a3534_38


In [30]:
os.makedirs('outputs',exist_ok=True)

In [31]:
#let's deploy since the automl is stronger thatn the hyperparametr model
from azureml.core.model import Model
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig


### Registering and Deployment (4th Try)

In [32]:
from azureml.core import Model
from azureml.core.webservice import AciWebservice, Webservice
import joblib

In [33]:
#best_automl_run
model = best_automl_run.register_model(model_name='best_run_automl.pkl', model_path='./outputs/',
                       model_framework=Model.Framework.SCIKITLEARN)

In [34]:
model

Model(workspace=Workspace.create(name='quick-starts-ws-137654', subscription_id='6971f5ac-8af1-446e-8034-05acea24681f', resource_group='aml-quickstarts-137654'), name=best_run_automl.pkl, id=best_run_automl.pkl:1, version=1, tags={}, properties={})

In [35]:
print(model.id)


best_run_automl.pkl:1


In [36]:
#Defining the deployment
from azureml.core.webservice import AciWebservice

In [37]:
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2,
                                              enable_app_insights=True)

                                              

In [38]:
service_name = 'my-sklearn-service'
service = Model.deploy(ws, service_name, [model], deployment_config=aci_config)

In [39]:
service.wait_for_deployment(show_output = True)
print(service.get_logs())

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running..................................................................
Failed


ERROR:azureml.core.webservice.webservice:Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 2600f536-1a8d-4ce0-a57f-24c6742231ea
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ImportError: cannot import name 'joblib' from 'sklearn.externals' (/azureml-envs/azureml_00066918e50f5ae481b726dc526556ed/lib/python3.7/site-packages/sklearn/externals/__init__.py), please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ImportError: cannot import name 'joblib' from 'sklearn.externals' (/azureml-envs/azureml_00066918e50f5ae481b726dc526556ed/lib/python3.7/site-packages/sklearn/externals/__init__.py), please run print(service.get_logs()) to get details."
    }
  ]
}



WebserviceException: WebserviceException:
	Message: Service deployment polling reached non-successful terminal state, current service state: Failed
Operation ID: 2600f536-1a8d-4ce0-a57f-24c6742231ea
More information can be found using '.get_logs()'
Error:
{
  "code": "AciDeploymentFailed",
  "statusCode": 400,
  "message": "Aci Deployment failed with exception: Error in entry script, ImportError: cannot import name 'joblib' from 'sklearn.externals' (/azureml-envs/azureml_00066918e50f5ae481b726dc526556ed/lib/python3.7/site-packages/sklearn/externals/__init__.py), please run print(service.get_logs()) to get details.",
  "details": [
    {
      "code": "CrashLoopBackOff",
      "message": "Error in entry script, ImportError: cannot import name 'joblib' from 'sklearn.externals' (/azureml-envs/azureml_00066918e50f5ae481b726dc526556ed/lib/python3.7/site-packages/sklearn/externals/__init__.py), please run print(service.get_logs()) to get details."
    }
  ]
}
	InnerException None
	ErrorRespo

In [40]:
print(service.get_logs())

2021-02-06T20:34:21,101174774+00:00 - iot-server/run 
2021-02-06T20:34:21,102648230+00:00 - gunicorn/run 
2021-02-06T20:34:21,103752272+00:00 - rsyslog/run 
2021-02-06T20:34:21,102967542+00:00 - nginx/run 
EdgeHubConnectionString and IOTEDGE_IOTHUBHOSTNAME are not set. Exiting...
2021-02-06T20:34:21,288396572+00:00 - iot-server/finish 1 0
2021-02-06T20:34:21,289722022+00:00 - Exit code 1 is normal. Not restarting iot-server.
Starting gunicorn 19.9.0
Listening at: http://127.0.0.1:31311 (12)
Using worker: sync
worker timeout is set to 300
Booting worker with pid: 39
SPARK_HOME not set. Skipping PySpark Initialization.
Initializing logger
2021-02-06 20:34:22,252 | root | INFO | Starting up app insights client
2021-02-06 20:34:22,253 | root | INFO | Starting up request id generator
2021-02-06 20:34:22,253 | root | INFO | Starting up app insight hooks
2021-02-06 20:34:22,253 | root | INFO | Invoking user's init function
Model path: azureml-models/best_run_automl.pkl/1/outputs
azureml-model

In [ ]:
import requests
import json
# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'

scoring_uri = ''
# If the service is authenticated, set the key or token
key = ''
# Two sets of data to score, so we get two results back
test_data = json.dumps({'data':[{
    'pclass': 0.8419164182590155,
    'age': -0.34907541344456255,
    'sibsp': -0.47908676070718687,
    'parch': -0.444999501816175,
    'fare': -0.4902404567566683,
    'age_NA': -0.5014319838391105,
    'fare_NA': -0.027650063180466557,
    'sex_male': 0.743496915331831,
    'cabin_Missing': 0.5393765119990418,
    'cabin_Rare': -0.42592011250734235,
    'embarked_Q': -0.32204029159373954,
    'embarked_Rare': -0.03911805059269843,
    'embarked_S': 0.6573935670276714,
    'title_Mr': 0.8525918887485938,
    'title_Mrs': -0.42592011250734235,
    'title_Rare': -0.27494677157229536
    }
    ]
        })

test_data2 = json.dumps({'data':[{
    'pclass': -15460978645168200,
    'age': 0.8912042887450313,
    'sibsp': -0.47908676070718687,
    'parch': -0.444999501816175,
    'fare': 19569900306355100,
    'age_NA': -0.5014319838391105,
    'fare_NA': -0.027650063180466557,
    'sex_male': -13449954927569300,
    'cabin_Missing': -18539924853119600,
    'cabin_Rare': 23478581326275300,
    'embarked_Q': -0.32204029159373954,
    'embarked_Rare': -0.03911805059269843,
    'embarked_S': -15211587854766800,
    'title_Mr': -11728941046668400,
    'title_Mrs': -0.42592011250734235,
    'title_Rare': -0.27494677157229536

    }
    ]
        })
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)
# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
headers['Authorization'] = f'Bearer {key}'
# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

### End 4th Try; ignore the code below...Therse are other tries

In [ ]:
#Generating the environment needed:
env = best_automl_run.get_environment()

In [ ]:
for pip_package in ["scikit-learn"]:
    env.python.conda_dependencies.add_pip_package(pip_package)

inference_config = InferenceConfig(entry_script='score.py',environment=env)

In [ ]:
#service = model.deploy(Workspace=ws,name = 'titanic best model',models = [model],inference_config = inference_config, deployment_config=aci_config,overwrite=True)

service_name = 'my-service-ml-3'

service = Model.deploy(ws, service_name, [model], inference_config, deployment_config=aci_config)

In [ ]:
#check on service
service

In [ ]:
#Check on deployment status:
service.wait_for_deployment(True)
print(service.state)


In [ ]:
#Now let's get the scoring URI:
scoring_uri = service.scoring_uri
scoring_uri

### The above below is not needed anymore; only the very next line will be then matched befores we submit the experiment

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(pipeline_run).show()

In [ ]:
pipeline_run.wait_for_completion()

## Examine Results

### Retrieve the metrics of all child runs
Outputs of above run can be used as inputs of other steps in pipeline. In this tutorial, we will examine the outputs by retrieve output data and running some tests.

In [ ]:
metrics_output = pipeline_run.get_pipeline_output(metrics_output_name)
metrics_output

In [ ]:
num_file_downloaded = metrics_output.download('.', show_progress=True)

In [ ]:
import json
with open(metrics_output._path_on_datastore) as f:
    metrics_output_result = f.read()
    
deserialized_metrics_output = json.loads(metrics_output_result)
df = pd.DataFrame(deserialized_metrics_output)
df

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve best model from Pipeline Run
best_model_output = pipeline_run.get_pipeline_output(best_model_output_name)
num_file_downloaded = best_model_output.download('.', show_progress=True)

In [ ]:
#TODO: Save the best model
import pickle

with open(best_model_output._path_on_datastore, "rb" ) as f:
    best_model = pickle.load(f)
best_model

In [ ]:
best_model.steps

### Test the Model:

In [ ]:
from sklearn.metrics import confusion_matrix
ypred = best_model.predict(x_test)
cm = confusion_matrix(y_test, ypred)

In [ ]:
pd.DataFrame(cm).style.background_gradient(cmap='Blues', low=0, high=0.9)

In [ ]:
#Accuracy Test:
(152 + 62) / (152 + 62 + 38+10)

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [ ]:
best_model

TODO: In the cell below, send a request to the web service you deployed to test it.

In [ ]:
#help(best_model)
dir(pipeline_run)

In [ ]:
pipeline_run.experiment

TODO: In the cell below, print the logs of the web service and delete the service